## **Overview :-**
This project uses **DistilBERT**, a smaller, faster, and lighter version of BERT, for detecting emotions from text. The model is fine-tuned on a text dataset and deployed as an interactive web application using **Streamlit**.


## **Dataset :-** https://huggingface.co/datasets/dair-ai/emotion

## **Importing Libraries**

In [ ]:
!pip install -U transformers
!pip install -U accelerate
!pip install -U datasets
!pip install -U bertviz
!pip install -U umap-learn

In [ ]:
import pandas as pd
from datasets import load_dataset
import matplotlib.pyplot as plt
from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import classification_report
import torch

In [ ]:
import pandas as pd
from datasets import load_dataset

ds = load_dataset("dair-ai/emotion", "split")

In [ ]:
ds

In [ ]:
ds.set_format(type='pandas')

## **Training Data**

In [ ]:
df = ds['train'][:]
df.head()

In [ ]:
ds['train']

In [ ]:
ds['train'].features

In [ ]:
classes = ds['train'].features['label'].names
classes

In [ ]:
df['label_name'] = df['label'].apply(lambda x: classes[x])

In [ ]:
df.head()

## **Validation and Test Data**

In [ ]:
df_valid = ds['validation'][:]
df_valid.head()

In [ ]:
df_valid.shape

In [ ]:
df_test = ds['test'][:]
df_test.head()

In [ ]:
df_test.shape

## **Dataset Analysis**

In [ ]:
df.info()

In [ ]:
df['label_name'].value_counts(ascending = True)

In [ ]:
df['label_name'].value_counts(ascending = True)/len(df)*100

In [ ]:
import matplotlib.pyplot as plt

label_counts = df['label_name'].value_counts(ascending=True)
label_counts.plot.barh()
plt.title('Frequency of Classes')
plt.show()

In [ ]:
df['Words Per Tweet'] = df['text'].str.split().apply(len)

In [ ]:
df.head()

In [ ]:
df['Words Per Tweet'].max()

In [ ]:
df.boxplot("Words Per Tweet", by='label_name')

## **Text to Tokens Conversion**

In [ ]:
from transformers import AutoTokenizer
model_ckpt = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

In [ ]:
tokenizer.vocab_size, tokenizer.model_max_length

## **Tokenization of the Emotion Data**

In [ ]:
ds.reset_format()

In [ ]:
ds

In [ ]:
def tokenize(batch):
  temp =tokenizer(batch['text'], padding=True, truncation=True)
  return temp

print(tokenize(ds["train"][:5]))

In [ ]:
emotions_encoded = ds.map(tokenize, batched=True, batch_size=None)

In [ ]:
emotions_encoded

In [ ]:
# Get the max length of tokenized inputs from the emotions_encoded dataset
max_length_train = max([len(item['input_ids']) for item in emotions_encoded['train']])
max_length_validation = max([len(item['input_ids']) for item in emotions_encoded['validation']])
max_length_test = max([len(item['input_ids']) for item in emotions_encoded['test']])

In [ ]:
max_length_train , max_length_validation , max_length_test

## **Model Building**

In [ ]:
model_ckpt

In [ ]:
from transformers import AutoModel
import torch

model = AutoModel.from_pretrained(model_ckpt)

In [ ]:
model

## **Fine-Tuning DistilBERT**

In [ ]:
from transformers import AutoModelForSequenceClassification

num_labels = len(classes)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AutoModelForSequenceClassification.from_pretrained(model_ckpt, num_labels = num_labels).to(device)

In [ ]:
device

In [ ]:
from transformers import TrainingArguments

In [ ]:
batch_size = 64
model_name = "distilbert-finetuned-emotion"

training_args = TrainingArguments(output_dir = model_name,
                                 #run_name="my-emotion-finetuning-run", # Add this line
                                 num_train_epochs=2,
                                 learning_rate = 2e-5,
                                 per_device_train_batch_size= batch_size,
                                 per_device_eval_batch_size = batch_size,
                                 weight_decay=0.01,
                                 #evaluation_strategy = "epoch",
                                 disable_tqdm=False)

In [ ]:
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  f1 = f1_score(labels, preds, average='weighted')
  acc = accuracy_score(labels, preds)
  return {"accuracy": acc, "f1": f1}

In [ ]:
from transformers import Trainer

trainer = Trainer(model=model, args=training_args,
                  compute_metrics=compute_metrics,
                  train_dataset=emotions_encoded['train'],
                  eval_dataset=emotions_encoded['validation'],
                  tokenizer=tokenizer)

In [ ]:
!pip install wandb -qU
import wandb
wandb.login()
# Before calling trainer.train()
wandb.init(project="my-emotion-finetuning-run", name="distilbert-finetuned-emotion")

# Now call the train method
trainer.train()

In [ ]:
preds_outputs = trainer.predict(emotions_encoded['test'])
preds_outputs.metrics

## **Saving the Model**

In [ ]:
trainer.save_model("./distilbert-finetuned-emotion-saved")

In [ ]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# Load the fine-tuned model and tokenizer
model_path = "./distilbert-finetuned-emotion-saved"
model = AutoModelForSequenceClassification.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Save the model and tokenizer with reduced precision (e.g., fp16)
# This significantly reduces the model size.
model.save_pretrained(model_path, max_shard_size="100MB", safe_serialization=True)
tokenizer.save_pretrained(model_path)

## **Loading the Model**

In [ ]:
loaded_model = AutoModelForSequenceClassification.from_pretrained(model_path)
loaded_tokenizer = AutoTokenizer.from_pretrained(model_path)

print(loaded_model)

## **Classification Report**

In [ ]:
import numpy as np
y_preds = np.argmax(preds_outputs.predictions, axis=1)
y_true = emotions_encoded['test'][:]['label']

In [ ]:
from sklearn.metrics import classification_report
print(classes)
print(classification_report(y_true, y_preds))

In [ ]:
label_counts

## **Prediction on Single Text**

In [ ]:
device

In [ ]:
text = 'i want to kill you'
input_encoded = tokenizer(text, return_tensors='pt')
# Move the input tensor to the same device as the model
input_encoded = input_encoded.to('cpu')
with torch.no_grad():
  outputs = model(**input_encoded)

logits = outputs.logits
pred = torch.argmax(logits, dim=1).item()
pred, classes[pred]

In [ ]:
outputs